# Song Mingi

## Import modules

In [34]:
import os
import csv
import torch
import wandb
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from torch import randperm
import torch.nn as nn

## Custom Dataset

In [82]:
import PIL.Image
import csv

class MyDataset(Dataset):

    def __init__(self, root: str = '~/', \
                transforms = None, \
                val_ratio:float = 0.0):
        
        self.path = os.path.join(root, 'input/data/train/images')
        self.transforms = transforms
        self.val_ratio = val_ratio
        self.dirs = next(os.walk(self.path))[1]
        self.images = {dir:[] for dir in self.dirs}
        for dir in self.dirs:
            dir_path = os.path.join(self.path ,dir)
            files = next(os.walk(dir_path))[2]
            files = [file for file in files if not file.startswith('._')]
            self.images[dir] = files
        self.classes = {'Mask': {'Wear': 0, 'Incorrect':1, 'Not Wear':2},
                'Gender': {'Male':0, 'Female':1},
                'Age': lambda x:x//30}

        self.data = self.get_data()
        self.write_csv()


    def get_label(self, dirname: str, filename:str) -> int:
        idx, gender, race, age = dirname.split('_')
        mask = ''
        if filename.startswith('normal'):
            mask = 'Not Wear'
        elif filename.startswith('incorrect_mask'):
            mask = 'Incorrect'
        else:
            mask = 'Wear'

        if gender == 'male':
            gender = 'Male'
        elif gender == 'female':
            gender = 'Female'

        label_num = self.classes['Mask'][mask] * 6 + self.classes['Gender'][gender] * 3 + self.classes['Age'](int(age))
        return label_num

    
    def write_csv(self):
        f = open('data.csv', 'w')
        wr = csv.writer(f)
        
        wr.writerow(('idx', 'dir', 'name', 'label'))

        for row in self.data:
            wr.writerow(row)
        
        f.close()

    
    def get_data(self):
        data = []
        idx = 0
        for dir in self.dirs:
            for file in self.images[dir]:
                data.append([idx, dir, file, self.get_label(dir, file)])
                idx += 1

        return data


    def read_images(self):
        images = {}
        for dir in self.dirs:
            dir_path = os.path.join(self.path ,dir)
            files = next(os.walk(dir_path))[2]
            files = [file for file in files if not file.startswith('._')]
            for file in files:
                images[dir] = {}
                images[dir][file] = {'label': self.get_label(dir, file)}
        return images
        
        
    def __getitem__(self, idx):
        _, dir, name, label = self.data[idx]
        img_path = os.path.join(self.path, dir, name)
        img = PIL.Image.open(img_path)
        if self.transforms:
            img = transforms(img)
        return img, label

    
    def __len__(self):
        return len(self.data)


    def __repr__(self):
        return '*' * 20 + '\n' + \
                'MyDataset class\n' + \
                f'path: {self.path}\n' + \
                f'transforms: {self.transforms}\n' + \
                f'dirs: {self.dirs[:30]}' + ('...\n' if len(self.dirs) > 30 else '\n') + \
                f'len: {len(self)}\n' + \
                '*' * 20



## Variables

In [36]:
root = '/opt/ml'
transforms = transforms.Compose([transforms.ToTensor(), transforms.CenterCrop(256)])
val_ratio = 0.2

In [37]:
ds = MyDataset(root, transforms, val_ratio)
print(ds)

********************
MyDataset class
path: /opt/ml/input/data/train/images
transforms: Compose(
    ToTensor()
    CenterCrop(size=(256, 256))
)
dirs: ['006108_male_Asian_19', '000262_male_Asian_56', '005107_female_Asian_38', '001439_male_Asian_58', '001047_male_Asian_60', '006753_male_Asian_19', '003619_male_Asian_57', '006452_female_Asian_18', '001362_female_Asian_28', '000750_female_Asian_52', '001725_female_Asian_25', '000334_female_Asian_52', '006093_male_Asian_19', '001495-1_male_Asian_21', '001282_female_Asian_18', '003004_female_Asian_19', '006164_female_Asian_19', '006102_male_Asian_18', '004242_female_Asian_53', '005530_female_Asian_19', '005028_male_Asian_52', '005051_female_Asian_26', '000661_male_Asian_55', '000827_female_Asian_55', '001479_female_Asian_57', '006075_male_Asian_18', '003307_female_Asian_19', '001454_female_Asian_49', '000270_female_Asian_49', '001019_male_Asian_29']...
len: 18900
********************


In [38]:
dl = DataLoader(ds, batch_size=64)

## Pre-trained Model

### vgg19

In [58]:
from torchvision.models import vgg19
from torch.nn.modules.linear import Linear

model = vgg19(pretrained=True)
last_output_features = model.classifier[-1].out_features
model.classifier.add_module(str(len(model.classifier)), Linear(in_features=last_output_features, out_features=18))
print(model.classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
  (7): Linear(in_features=1000, out_features=18, bias=True)
)


In [43]:
from copy import deepcopy

def train_model(model, dataloader, device, criterion, optimizer, num_epochs):
    best_model = None
    best_acc = 0.0

    for epoch in range(1, num_epochs+1):
        print('=' * 20)
        print(f'Epoch: {epoch}/{num_epochs}')

        running_loss = {'train': 0, 'val': 0}
        running_corrects = {'train': 0, 'val': 0}

        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)
            loss = criterion(outs, labels)

            loss.backward()
            optimizer.step()
            
            running_loss['train'] += loss.item() * inputs.size(0)
            running_corrects['train'] += torch.sum(preds == labels.data)

        train_loss = running_loss['train'] / len(dl.dataset)
        train_acc = running_corrects['train'] / len(dl.dataset)
        # val_loss = running_loss[mode] / len(dl.dataset.indicies['val'])
        # val_acc = running_corrects[mode] / len(dl.dataset.indicies['val'])

        print(f'train loss: {train_loss}, train acc: {train_acc}')
        # print(f'val loss: {val_loss}, val acc: {val_acc}')
        wandb.log({"train": {"loss": train_loss, "acc": train_acc}})
        # wandb.log({"val": {"loss": val_loss, "acc": val_acc}})

        if best_acc < train_acc:
            train_acc = best_acc
            best_model = deepcopy(model.state_dict())

    return best_model

In [41]:
from torch import optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
model.cuda()

print(f'device: {device}')

device: cuda:0


In [44]:

wandb.init(project="my project", entity="songmingi")
best_model = train_model(model, dl, device, criterion, optimizer_ft, 3)
torch.save(model, '/opt/ml/model.pt')


/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
2022-02-24 04:25:41.285632: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: l

Epoch: 1/3
train loss: 0.9573197424664068, train acc: 0.6842328310012817
Epoch: 2/3
train loss: 0.4755650469991896, train acc: 0.8419047594070435
Epoch: 3/3
train loss: 0.3478045103474269, train acc: 0.880740761756897


In [51]:
eval_path = '/opt/ml/input/data/eval/images'
eval_images = next(os.walk(eval_path))[2]
eval_images = [image for image in eval_images if not image.startswith('._')]

print(f'eval images number: {len(eval_images)}')

eval images number: 12600


In [60]:
img_path = os.path.join(eval_path, eval_images[0])
img = PIL.Image.open(img_path)
print(type(img))

<class 'PIL.JpegImagePlugin.JpegImageFile'>


In [64]:
model.load_state_dict(torch.load('/opt/ml/model.pt'))

<All keys matched successfully>

In [81]:
import pandas as pd
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from PIL import Image


class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)


test_dir = '/opt/ml/input/data/eval'

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
model.eval()
device = torch.device('cuda')
model.to(device)


# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

test inference is done!
